In [1]:
import os
import re

In [2]:
# just once, no need to run again
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 54992 (delta 119), reused 220 (delta 107), pack-reused 54748
Receiving objects: 100% (54992/54992), 570.96 MiB | 2.99 MiB/s, done.
Resolving deltas: 100% (37775/37775), done.


In [6]:
# just once, no need to run again
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/Users/paulefeb/Projects/object_detection_modeling/models/research
Processing /Users/paulefeb/Projects/object_detection_modeling/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1632827 sha256=f6d1f9679b2e15d3700e580a0e2bafffb7d8177fca3e87f150718a2e76c8d37c
  Stored in directory: /private/var/folders/lg/0m539gbj15z1szhfdfx2fqfh0000gn/T/pip-ephem-wheel-cache-v23q5xky/wheels/a8/1e/6f/458863e99993c54e56eb10f7b0d2204cacc2345053c59d3f40
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [7]:
%cd ../..

/Users/paulefeb/Projects/object_detection_modeling


In [8]:
# Run the model builder test
!python models/research/object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.8.5: /Users/paulefeb/opt/anaconda3/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2021-03-31 22:44:25.918488: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-31 22:44:25.919818: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model): 0.67s
I0331 22:44:26.228069 4593724864 test_util.py:2075] time(__main__.ModelBuilderTF2Test.test_create_center_net_model): 0.67s
[       OK ] ModelBuilderTF2Test.test_create_center_net_model
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model_from_keypoints
INFO:tensorflow:time(__main__.ModelB

In [17]:
pwd

'/Users/paulefeb/Projects/object_detection_modeling'

In [18]:
!python dataset/generate_tf_records.py -l labelmap.pbtxt -o dataset/train.record -i dataset/images -csv train_labels.csv
!python dataset/generate_tf_records.py -l labelmap.pbtxt -o dataset/test.record -i dataset/images -csv test_labels.csv

INFO:Successfully created the TFRecords: dataset/train.record
INFO:Successfully created the TFRecords: dataset/test.record


In [19]:
pwd

'/Users/paulefeb/Projects/object_detection_modeling'

In [32]:
num_classes = 1
batch_size = 96
num_steps = 5000
num_eval_steps = 1000

train_record_path = '/Users/paulefeb/Projects/object_detection_modeling/dataset/train.record'
test_record_path = '/Users/paulefeb/Projects/object_detection_modelingdataset/test.record'
model_dir = 'training/'
labelmap_path = 'labelmap.pbtxt'

pipeline_config_path = 'mobilenet_v2.config'
fine_tune_checkpoint = 'mobilenet_v2/mobilenet_v2.ckpt-1'

In [33]:
with open(pipeline_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:

  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  f.write(config)

In [34]:
!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

2021-03-31 23:56:31.371073: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-31 23:56:31.371304: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
W0331 23:56:31.372151 4816043456 cross_device_ops.py:1321] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0331 23:56:31.372274 4816043456 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 5000
I0331 23:56:31.375213 4816043456 config_util.py:552] Maybe overwriting train_steps:

In [ ]:
# Once the evaluation finishes, you're going to see the message:
# INFO:tensorflow:Waiting for new checkpoint at /training/
# Then you can stop the cell

!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir}

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/training/'

In [ ]:
with open('/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/tf_utils.py') as f:
    tf_utils = f.read()

with open('/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/tf_utils.py', 'w') as f:
  # Set labelmap path
  throw_statement = "raise TypeError('Expected Operation, Variable, or Tensor, got ' + str(x))"
  tf_utils = tf_utils.replace(throw_statement, "if not isinstance(x, str):" + throw_statement)
  f.write(tf_utils)

In [ ]:
output_directory = 'inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

In [ ]:
!zip -r /content/saved_model.zip /content/inference_graph/saved_model/

In [ ]:
from google.colab import files
files.download("/content/saved_model.zip")